In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch

#### Creamos nuestra clase de datos:

In [3]:
links_csv = ['https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/train1.csv',
'https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/train2.csv',
'https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/train3.csv',
'https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/train4.csv',]

class MNISTDataset(Dataset):
  def __init__(self, links_csv, mode='train'):
    self.mode = mode #
    list_df = [pd.read_csv(link) for link in links_csv]
    data = pd.concat(list_df)

    """
    <---------------------------- Preprocesamiento de Datos -------------------------------------->
    """

    if self.mode == 'train':
      self.features = data.drop('label',axis=1).values/255.0
      self.target = data['label'].values
      #Reshaping
      self.target = self.target.reshape((-1,1)) #Dejando q se reeshapee solo

    else:
      self.features = data.values/255.0

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    if self.mode=='train':
      feat = torch.Tensor(self.features[idx])
      oupt = torch.Tensor(self.target[idx])

      return {'features':feat,
              'target':oupt,}
      
    else:
      feat = torch.Tensor(self.features[idx])
      return {'features':feat}

In [4]:
test_links = ['https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/test.csv']

train = MNISTDataset(links_csv = links_csv, mode='train')
test = MNISTDataset(links_csv=test_links, mode='test')

In [5]:
#Ahora vamos a crear los DataLoades para poder obtener los batchs.

train_loader = DataLoader(train, batch_size=64, shuffle = True)
test_loader = DataLoader(test, batch_size=64, shuffle =True)

### Modelling MultiLayerPerceptron (MLP)

Vamos a ver que primero haremos una clase heredada de la clase nn.Module, y también usaremos super() para poder heredar los atributos de la clase nn.Module. Así tendremos acceso y más flexibilidad para extender esta clase nn.Module. 

#### Definiendo el modelo:

In [26]:
import torch.nn as nn

class MLP(nn.Module):
  def __init__(self, i, u, v, o):
    """
    Donde i,u,v,o son los números de neuronas de 
    entrada y salida que tendrá mi red.
    """
    super(MLP, self).__init__()
    self.relu_layer = nn.ReLU()
    self.dense_1 = nn.Linear(i, u)
    self.b1 = nn.BatchNorm1d(20)
    self.dense_2 = nn.Linear(u,v)
    self.dense_output = nn.Linear(v, o)


  def forward(self, x):
    x = self.relu_layer(self.dense_1(x)) # activation( Capa )
    x = self.b1(x) #Batch Layer
    x = self.relu_layer(self.dense_2(x)) # Activation(Capa)

    logits = self.dense_output(x)

    return logits

### Inicializando la red:

In [27]:
from torch.optim import Adam

#Usamos el GPU
device = torch.device('cuda')
red_neuronal = MLP(i=784, u = 20, v=20, o = 10).to(device) #Acá puedo definir los layers <- Notar nombre de objeto.
optimizer = Adam(params=red_neuronal.parameters(), lr=0.01)
print(red_neuronal) #Imprimimos la arquitectura

MLP(
  (relu_layer): ReLU()
  (dense_1): Linear(in_features=784, out_features=20, bias=True)
  (b1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense_2): Linear(in_features=20, out_features=20, bias=True)
  (dense_output): Linear(in_features=20, out_features=10, bias=True)
)


In [28]:
#Definimos nuestra función de pérdida

def accuracy(y_true, y_pred):
  y_true = y_true.long().squeeze()
  y_pred = torch.argmax(y_pred, axis=1)
  return (y_true == y_pred).float().sum()/len(y_true)

def cross_entropy_loss(y_true, y_pred):
  y_true = y_true.long().squeeze()
  return nn.CrossEntropyLoss()(y_pred, y_true)

### Entrenando

In [30]:
#Esto también usualmente se pone dentro de una función: training_loop.

import time
start = time.time()
print("STARTING TRAINING ...\n")

train_losses, valid_losses = [], []
train_acc, valid_acc = [], []

for epoch in range(20): #Definiendo 20 epocas.
  red_neuronal = red_neuronal.train()
  print("Epoch: {}".format(epoch+1))
  batch_train_losses, batch_train_acc = [], []

  batch = 0
  for train_batch in train_loader: #Objeto DataLoader

    train_X, train_y = train_batch.values()

    train_X = train_X.to(device)
    train_y = train_y.to(device)

    train_preds = red_neuronal.forward(train_X)
    train_loss = cross_entropy_loss(train_y, train_preds)

    train_accuracy = accuracy(train_y, train_preds)

    optimizer.zero_grad() # ????
    train_loss.backward() #Back prop?

    optimizer.step() #

    train_loss = np.round(train_loss.item(), 3)
    train_accuracy = np.round(train_accuracy.item(),3)

    end = time.time()
    batch = batch + 1
    log = batch % 10 == 0

    time_delta = np.round(end - start, 3)

    batch_train_losses.append(train_loss)
    batch_train_acc.append(train_accuracy)

    logs = "Batch: {} || Train Loss: {} || Train Acc: {} || Time: {} s"
    if log: print(logs.format(batch, train_loss, train_acc, time_delta))

  #Hasta acá solo hemos entrenado. Aún no evaluamos con respecto al test. 

  train_losses.append(np.mean(batch_train_losses))
  train_acc.append(np.mean(batch_train_acc))

STARTING TRAINING ...

Epoch: 1
Batch: 10 || Train Loss: 1.449 || Train Acc: [] || Time: 0.038 s
Batch: 20 || Train Loss: 1.0 || Train Acc: [] || Time: 0.073 s
Batch: 30 || Train Loss: 0.662 || Train Acc: [] || Time: 0.11 s
Batch: 40 || Train Loss: 0.548 || Train Acc: [] || Time: 0.147 s
Batch: 50 || Train Loss: 0.522 || Train Acc: [] || Time: 0.182 s
Batch: 60 || Train Loss: 0.558 || Train Acc: [] || Time: 0.221 s
Batch: 70 || Train Loss: 0.434 || Train Acc: [] || Time: 0.256 s
Batch: 80 || Train Loss: 0.341 || Train Acc: [] || Time: 0.295 s
Batch: 90 || Train Loss: 0.518 || Train Acc: [] || Time: 0.329 s
Batch: 100 || Train Loss: 0.637 || Train Acc: [] || Time: 0.363 s
Batch: 110 || Train Loss: 0.181 || Train Acc: [] || Time: 0.396 s
Batch: 120 || Train Loss: 0.383 || Train Acc: [] || Time: 0.433 s
Batch: 130 || Train Loss: 0.466 || Train Acc: [] || Time: 0.469 s
Batch: 140 || Train Loss: 0.325 || Train Acc: [] || Time: 0.504 s
Batch: 150 || Train Loss: 0.313 || Train Acc: [] || Time

In [17]:
a, o = train_batch.values()
a

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])